### 해당 모델 관련한 내용

**GBT (Gradient Boosting Tree)**
- 부스팅 기법의 하나인 GBM(gradient boosting machine)을 결정 트리에 적용한 앙상블 모델
- 회귀, 분류에 모두 적용 가능
- 이전 단계에서 학습하지 못한 잔차를 새로운 목표 변수로 두고 학습  
$y \leftarrow h_1(x)$  
$y - h_1(x) \leftarrow h_2(x)$  
$y - h_1(x) - h_2(x) \leftarrow h_3(x)$  
$y - \displaystyle\sum_{m=1}^{M-1} h_m(x) \leftarrow h_M(x)$
$$\hat{y} = \displaystyle\sum_{m=1}^{M}h_m(x)$$
- 이 아이디어에 기반하여, 실제 최적화 된 해를 계산할 때는 경사 하강법을 이용함
---
1. GBT 회귀
- 직렬로 학습하는 가산 모델
- 그리디 알고리즘 방식으로 진행
- 손실함수는 제곱 오차 손실 또는 절대 오차 손실로 정의된다.
---
2. GBT 분류
- GBT 회귀 모델에서의 예측값 $F_m(x_i) = \sum h_m(x_i)$를 클래스 또는 클래스의 확률 값으로 변환하는 과정을 추가로 진행해야 한다.
- 변환 방법은 손실 함수에 따라 달라진다.
- 약학 학습기 $h_m$은 분류 모델이 아니라 여전히 회귀 모델이라는 사실을 명심해야 한다.
    - 이는 예측하고자 하는 값이 연속값인 그레이디언트기 때문!

In [3]:
# 부스팅 트리 회귀 모델 구현하기
from sklearn.datasets import load_diabetes
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import numpy as np

df = load_diabetes(as_frame=True)['frame']
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.33,
                                                   random_state=1234)

In [5]:
y_train2 = y_train.copy()

trees = []
for i in range(100):
    reg = DecisionTreeRegressor(max_depth=4,
                               random_state=1234).fit(X_train, y_train2)
    y_pred = reg.predict(X_train)
    y_train2 = y_train2 - y_pred
    trees.append(reg)

y_pred = np.zeros(len(y_test))
for tree in trees:
    y_pred += tree.predict(X_test)

print(f'테스트 데이터셋 기준 MAE : {(np.abs(y_pred - y_test)).mean():.3f}')

테스트 데이터셋 기준 MAE : 54.282


In [7]:
# GradienBosstingRegressor 클래스로 모델 성능 평가
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(n_estimators=100,
                               max_depth=4,
                               random_state=1234)

y_pred = reg.fit(X_train, y_train).predict(X_test)

print(f'GBT 테스트 데이터셋 기준 MAE : {(np.abs(y_pred - y_test)).mean():.3f}')

y_pred = DecisionTreeRegressor(random_state=1234).fit(X_train, y_train).predict(X_test)
print(f'결정트리 테스트 데이터셋 기준 MAE :{(np.abs(y_pred - y_test)).mean():.3f}')

GBT 테스트 데이터셋 기준 MAE : 45.110
결정트리 테스트 데이터셋 기준 MAE :59.171


### 해당 모델 파라미터 내용  

1. 주요 하이퍼 파라미터
- loss : 손실 함수 선택
    - 'squared_error' : 제곱 오차 손실 사용
    - 'absolute_error' : 절대 오차 손실 사용
    - 'huber' : Huber 손실 사용 (L1과 L2의 장점을 합친 손실함수)
    - 'quantile' : 분위수 손실 사용
- learning_rate : 모든 규제를 위해 각 부스팅 단계에 적용하는 학습률
- n_estimators : 부스팅 단계의 수
- subsample : 모델의 규제를 위해 각 트리를 만들 때 전체 데이터 대신 subsample의 비율만을 랜덤 샘플링 하여 사용.
---
2. 조기 종료 로직
- validation_fraction : 조기 종료를 위해 사용할 검증 데이터셋 비율
- n_iter_no_change : 조기 종료를 적용하고자 모니터링할 이터레이션의 횟수
- tol : 조기 종료를 위한 허용 오차 (float)
---
3. 분류 모델에서 loss 하이퍼파라미터
- loss : 손실함수 선택
    - 'log_loss' : defualt
    - 'exponential'
---
4. 실제 활용 팁
- 모델 복잡도가 증가하는 n_estimators와 모델을 규제하는 learning_rate, subsample 등 상충관계에 있으므로 적절히 선택해야 한다.
- 일반적으로 성능에 가장 영향을 끼치는 중요한 하이퍼파라미터 **우선 고려**
    - n_estimators
    - learning_rate

### 해당 모델 관련 예시 코드 ( 타이타닉 ,,,등등)

In [9]:
# GBT회귀 클래스 적용하기
from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

df = load_diabetes(as_frame=True)['frame']
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.33,
                                                   random_state=1234)

In [10]:
reg = GradientBoostingRegressor(n_estimators=200,
                               learning_rate=0.05,
                               max_depth=5,
                               random_state=1234).fit(X_train, y_train)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5

rmse

54.318624016786984

In [15]:
reg = GradientBoostingRegressor(n_estimators=100,
                               learning_rate=0.05,
                               max_depth=5,
                               random_state=1234
                               ).fit(X_train, y_train)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5

rmse

54.20052077398064

### 적고싶은 내용 

- 공부하다가 이해 안되는 내용..등등